## panel app for diffusers img2img

In [1]:
import os, glob
import random, math
import numpy as np
import pandas as pd
import matplotlib as mpl
from PIL import Image
from tools import *

import panel as pn
import panel.widgets as pnw
pn.extension('tabulator', css_files=[pn.io.resources.CSS_URLS['font-awesome']])


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [2]:
def random_image(outfile):
    from randimage import get_random_image, show_array
    img_size = (256,256)
    img = get_random_image(img_size)
    show_array(img)
    mpl.image.imsave(outfile, img)

In [3]:
artists = list(pd.read_csv('stable_diffusion_artist_styles.csv').Name)
artists = ['','random']+artists

In [22]:
def dashboard():
    
    def run(**kwargs):
        """Run prompt""" 
      
        filename = img2imgprompt(path='temp', n=1, **kwargs)
        name = kwargs['seed']
        add_image(filename)
        #print (filename)

    w=210
    startimg = None
    styles = ['','oil','impressionist','pencil','ink','watercolor','crayon drawing','digital art','pop art','cubism',
              'sculpture','craft clay','linocut','engraving','anime','studio photography','analog film',
              'abstract','pixel art','paper collage','isometric','lowpoly','origami']
    
    #title = html_pane = pn.pane.HTML("""<h2>image-to-image app</h2>""")
    file_input = pnw.FileInput(width=w,accept='.png,.jpg')    
    go_btn = pnw.Button(name='run',width=w,button_type='success')
    stop_btn = pnw.Button(name='stop',width=w,button_type='danger')
    prompt_input = pnw.TextAreaInput(name='prompt',value='',width=w)
    style_input = pnw.Select(name='style',options=styles,width=w)
    artist_input = pnw.Select(name='artist',options=artists,width=w)
    strength_input = pnw.FloatSlider(name='strength',value=.8,step=.01,start=.01,end=.99,width=w)
    guidance_input = pnw.IntSlider(name='guidance',value=5,step=1,start=0,end=10,width=w)
    seed_input = pnw.IntInput(name='seed',value=0,step=1,start=0,end=10000,width=w)
    progress = pn.indicators.Progress(name='Progress', value=0, width=w, bar_color='primary')    
    widgets = pn.Column(pn.WidgetBox(file_input,prompt_input,style_input,artist_input,strength_input,
                                     guidance_input,seed_input,go_btn,progress), height=700, width=230)
                
    img_pane = pn.pane.Image(caption='template', sizing_mode="stretch_width")
    randimg_btn = pnw.Button(name='random template',width=160,button_type='primary')
    setcurrent_btn = pnw.Button(name='use current',width=160,button_type='primary')
    tabs = pn.Tabs(closable=True, tabs_location='left',sizing_mode="stretch_width")

    def random_template(event):
        random_image('randimg.png')
        img_pane.object = 'randimg.png'
        #file_input.filename = None
        
    def load_template(event):
        img_pane.object = file_input.value

    def current_to_template(event):
        img_pane.object = tabs[tabs.active].object
        #file_input.filename = None
        
    def add_image(imgfile):
        #add new image         
        name = os.path.basename(imgfile)
        new = pn.pane.Image(imgfile, caption=name, sizing_mode="stretch_both")        
        tabs.append(new)
        tabs.active = len(tabs)-1
              
    def execute(event):
        #run the model with widget         
        startimg = file_input.filename
        #print (file_input.filename,img_pane.object)
              
        if startimg == None:
            startimg = img_pane.object
        seed = seed_input.value
        if seed == 0:
            seed = None
        if startimg is None:
            return
        progress.value=-1
        if artist_input.value == 'random':
            artist = random.choice(artists[2:])
        else:
            artist = artist_input.value
        prompt = prompt_input.value+', '+style_input.value
        run(prompt=prompt, style=artist, 
            init_images=[startimg],strength=strength_input.value, seed=seed)
        progress.value=0
        
    file_input.param.watch(load_template, 'value')
    go_btn.param.watch(execute, 'clicks')
    randimg_btn.param.watch(random_template, 'clicks')
    setcurrent_btn.param.watch(current_to_template, 'clicks')
    
    app = pn.Column(
                 pn.Row(widgets,tabs,pn.Column(randimg_btn,setcurrent_btn,img_pane,height=500,width=220),
                 sizing_mode='stretch_both',
                 styles={'background': 'WhiteSmoke'}))

    return app

In [23]:
dashboard()

Column
    [0] Row(sizing_mode='stretch_both', styles={'background': '...})
        [0] Column(height=700, sizing_mode='fixed', width=230)
            [0] WidgetBox
                [0] FileInput(accept='.png,.jpg', width=210)
                [1] TextAreaInput(name='prompt', width=210)
                [2] Select(name='style', options=['', 'oil', 'impressionist...], width=210)
                [3] Select(name='artist', options=['', 'random', ...], width=210)
                [4] FloatSlider(end=0.99, name='strength', start=0.01, step=0.01, value=0.8, width=210)
                [5] IntSlider(end=10, name='guidance', value=5, width=210)
                [6] IntInput(end=10000, name='seed', start=0, width=210)
                [7] Button(button_type='success', name='run', width=210)
                [8] Progress(bar_color='primary', value=0, width=210)
        [1] Tabs(closable=True, sizing_mode='stretch_width', tabs_location='left')
        [2] Column(height=500, sizing_mode='fixed', width=220)
            [0] Button(button_type='primary', name='random template', width=160)
            [1] Button(button_type='primary', name='use current', width=160)
            [2] Image(None, caption='template', sizing_mode='stretch_width')